In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sys
sys.path.insert(0, '..')
import bundle as bd
from pyclk import Trace

%matplotlib inline

In [ ]:
ctrl_nb = 3
iter_nb = 4
mem_nb = iter_nb * 3 * 2
mem_depth = 1024
add_nb = iter_nb
mul_nb = iter_nb

fpga = bd.FPGA(ctrl_nb, iter_nb, mem_nb, mem_depth, add_nb, mul_nb)

In [ ]:
#fpga.set_cycle_nb(1000)

if True:
    trace = Trace()
    for i in range(fpga.config['mem_nb']):
        trace.add(fpga.u_mem[i].i_addr)
        trace.add(fpga.u_mem[i].o_dout)
        trace.add(fpga.u_mem[i].i_din)
        trace.add(fpga.u_mem[i].i_wena)
    for i in range(fpga.config['iter_nb']):
        trace.add(fpga.u_iter[i].o_wena)
        #trace.add(fpga.u_iter[i].o_done)
        #trace.add(fpga.u_iter[i].i_ack)
        #trace.add(fpga.u_iter[i].i_data_nb)
    for i in range(fpga.config['func_nb']):
        trace.add(fpga.u_func[i].o_res)
    fpga.set_trace(trace)

In [ ]:
size = 10_000
a0 = np.arange(size)
a1 = np.ones_like(a0)
a2 = np.zeros_like(a0)
expr = '(a0 + a1 * (a0 + a2)) * a1 + a1'

In [ ]:
res = bd.evaluate(expr)

In [ ]:
res

In [ ]:
# FPGA runs at 100 MHz
print('In FPGA:')
print(f'{fpga.time * 1e6 / 1e8} us')

In [ ]:
cexpr = compile(expr, 'fakemodule', 'exec')
print('In CPU:')
%timeit exec(cexpr)

In [ ]:
#res_true = bd.evaluate(expr, toFpga=False)
res_true = eval(expr)
res_true

In [ ]:
if np.array_equal(res, res_true):
    print('Success!')
else:
    print('Failed!')

In [ ]:
res = np.array(res)
diff_idx = np.where(res != res_true)
plt.figure(figsize=(15, 5))
plt.scatter(np.arange(res_true.size), res_true, c='b')
plt.scatter(diff_idx, res[diff_idx], c='r', label='FPGA')
plt.scatter(diff_idx, res_true[diff_idx], c='g', label='NumPy')
#plt.scatter(diff_idx, z_true[diff_idx]-z[diff_idx], c='r')
plt.xlim(0, size)
plt.ylim(1, size+1)
plt.legend(loc='upper left')
plt.title('Differences')
plt.show()

In [ ]:
res[diff_idx]

In [ ]:
res_true[diff_idx]

In [ ]:
trace.plot()